<a href="https://colab.research.google.com/github/Milafreire/web_scrap_wh_apache_beam_airflow/blob/main/web_scrap_extract_pipe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Configs

In [3]:
!pip install requests beautifulsoup4

In [4]:
#requirements
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
from datetime import datetime

In [5]:
# Google Auth
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

# GCP Project
os.environ["GOOGLE_CLOUD_PROJECT"]= 'mywebscrap-423316'

Authenticated


In [6]:
table_storage='gs://myscraptest/parquet_files/scrap_data'
initial_url = 'https://books.toscrape.com/catalogue/page-1.html'

##Methods

In [7]:
def get_book_details(book, book_id):
    title = book.find('h3').find('a').get('title')
    link = book.find('h3').find('a').get('href')
    details_url = f'https://books.toscrape.com/catalogue/{link}'

    book_response = requests.get(details_url)
    book_soup = BeautifulSoup(book_response.text, 'html.parser')

    upc = book_soup.find('th', text='UPC').find_next_sibling('td').text
    price_text = book_soup.find('th', text='Price (excl. tax)').find_next_sibling('td').text
    price = float(price_text.replace('£', '').replace('Â', '').strip())
    stock_text = book_soup.find('th', text='Availability').find_next_sibling('td').text
    stock = int(stock_text.split(' ')[2].strip('()'))
    category = book_soup.find('ul', class_='breadcrumb').find_all('a')[2].text
    rating_class = book_soup.find('p', class_='star-rating')['class']
    rating = rating_class[1] if len(rating_class) > 1 else 'Zero'
    reviews = int(book_soup.find('th', text='Number of reviews').find_next_sibling('td').text)
    extraction_date = datetime.now().strftime('%Y-%m-%d')

    return {
        'id': book_id,
        'bar_code': upc,
        'product_name': title,
        'category': category,
        'price': price,
        'qty_stock': stock,
        'star_rating': rating,
        'extraction_date': extraction_date
    }

In [8]:
def scrape_books(base_url):
    all_books = []
    url = base_url
    book_id = 1

    while url:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        books = soup.find_all('article', class_='product_pod')

        for book in books:
            book_details = get_book_details(book, book_id)
            all_books.append(book_details)
            book_id += 1

        next_link = soup.find('li', class_='next')
        url = f"https://books.toscrape.com/catalogue/{next_link.find('a')['href']}" if next_link else None

    return pd.DataFrame(all_books)

##Storage

In [9]:
books_df = scrape_books(initial_url)
books_df.to_parquet(table_storage)

<ipython-input-7-6555d57d3857>:9: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  upc = book_soup.find('th', text='UPC').find_next_sibling('td').text
<ipython-input-7-6555d57d3857>:10: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  price_text = book_soup.find('th', text='Price (excl. tax)').find_next_sibling('td').text
<ipython-input-7-6555d57d3857>:12: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  stock_text = book_soup.find('th', text='Availability').find_next_sibling('td').text
<ipython-input-7-6555d57d3857>:17: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  reviews = int(book_soup.find('th', text='Number of reviews').find_next_sibling('td').text)


##Viewing the dataframe

In [10]:
df = pd.read_parquet('gs://myscraptest/parquet_files/scrap_data')
print(df)

       id          bar_code  \
0       1  a897fe39b1053632   
1       2  90fa61229261140a   
2       3  6957f44c3847a760   
3       4  e00eb4fd7b871a48   
4       5  4165285e1663650f   
..    ...               ...   
995   996  cd2a2a70dd5d176d   
996   997  bfd5e1701c862ac3   
997   998  19fec36a1dfb4c16   
998   999  f684a82adc49f011   
999  1000  228ba5e7577e1d49   

                                          product_name            category  \
0                                 A Light in the Attic              Poetry   
1                                   Tipping the Velvet  Historical Fiction   
2                                           Soumission             Fiction   
3                                        Sharp Objects             Mystery   
4                Sapiens: A Brief History of Humankind             History   
..                                                 ...                 ...   
995  Alice in Wonderland (Alice's Adventures in Won...            Classics   
996